## Lista 5

In [183]:
from pyspark import SparkConf
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
import numpy as np
import networkx as nx
import pandas as pd
from graphframes import *

In [120]:
config = SparkConf().setAppName('appName').setMaster('local')
sctext = SparkContext.getOrCreate(config)
spark = SparkSession(sctext)

In [142]:
from graphframes import *

No = spark.read.csv('transport-nodes.csv', header=True)
relation = spark.read.csv('transport-relationships.csv', header=True)
graph = GraphFrame(No, relation)

### Crie um CSV para armazenar as cidades, com id (nome da cidade), latitude, longitude e população.

In [145]:
No.show()

+--------------------+---------+---------+----------+
|                  id| latitude|longitude|population|
+--------------------+---------+---------+----------+
|         João Pessoa|52.379189| 4.899431|    809015|
|      Campina Grande|52.092876| 5.104480|    334176|
|               Patos|52.078663| 4.288788|    107605|
|            Cabedelo| 53.61239| -0.22219|      9642|
|           São Bento| 53.52285| -1.13116|     34031|
|             Triunfo|  51.9775|  4.13333|      9382|
|          Mamanguape| 51.96375|   1.3511|     44882|
|          Itaporanga| 52.05917|  1.15545|     24692|
|               Areia| 51.88921|  0.90421|     22819|
|           Rio Tinto|51.509865|-0.118092|     24172|
|São José do rio d...|  51.9225|  4.47917|     18034|
|          São Mamede| 52.01667|  4.70833|      7724|
+--------------------+---------+---------+----------+



### Crie outro CSV para armazenar a distância entre essas cidades, com src, dst e relationship como campos (adicione pelo menos 30 registros nesse arquivo).

In [123]:
relation.show()

+--------------------+--------------------+------------+----+
|                 src|                 dst|relationship|cost|
+--------------------+--------------------+------------+----+
|         João Pessoa|      Campina Grande|       EROAD|  42|
|         João Pessoa|               Patos|       EROAD|  56|
|               Patos|            Cabedelo|       EROAD|  19|
|         João Pessoa|           São Bento|       EROAD| 297|
|           São Bento|          São Mamede|       EROAD|  35|
|          São Mamede|             Triunfo|       EROAD| 209|
|          Mamanguape|               Patos|       EROAD|  17|
|               Areia|          Mamanguape|       EROAD|  75|
|          Itaporanga|               Areia|       EROAD| 132|
|           Rio Tinto|          Itaporanga|       EROAD|  43|
|             Triunfo|           Rio Tinto|       EROAD|  85|
|São José do rio d...|            Cabedelo|       EROAD|  23|
|São José do rio d...|      Campina Grande|       EROAD|  32|
|       

## Utilizando as bibliotecas do Spark, crie um objeto GraphFrame a partir desses dois CSVs.

In [124]:
# Utilizando as bibliotecas do Spark, crie um objeto GraphFrame a partir desses dois CSVs.

# Número de relações entre os nós
graph.degrees.show()

+--------------------+------+
|                  id|degree|
+--------------------+------+
|               Areia|     3|
|            Cabedelo|     4|
|           São Bento|     3|
|          São Mamede|     4|
|         Itaporanga |     1|
|             Triunfo|     2|
|         João Pessoa|     6|
|           Rio Tinto|     3|
|São José do rio d...|     5|
|               Patos|     5|
|      Campina Grande|     4|
|          Mamanguape|     3|
|          Itaporanga|     5|
+--------------------+------+



## Utilizando o método bfs (Breadth First Search), execute 5 filtragens a sua escolha.

In [125]:
# Método 1

graph.bfs(fromExpr = "id = 'Areia'", toExpr = "id = 'Itaporanga'", edgeFilter = "relationship = 'EROAD'").show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                from|                  e0|                  v1|                  e1|                  v2|                  e2|                  v3|                  e3|                  v4|                  e4|                  v5|                  e5|                  v6|                  e6|                  to|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|[Areia, 51.88921,...|[Areia, Mamanguap...|[Maman

In [126]:
# Método 2

graph.bfs(fromExpr = "id = 'Triunfo'", toExpr = "id = 'Patos'", edgeFilter = "relationship = 'EROAD'").show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                from|                  e0|                  v1|                  e1|                  v2|                  e2|                  v3|                  e3|                  v4|                  e4|                  to|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|[Triunfo, 51.9775...|[Triunfo, Rio Tin...|[Rio Tinto, 51.50...|[Rio Tinto, Itapo...|[Itaporanga, 52.0...|[Itaporanga, Arei...|[Areia, 51.88921,...|[Areia, Mamanguap...|[Mamanguape, 51.9...|[Mamanguape, Pato...|[Patos, 52.078663...|
+--------------------+--------------------+--------------------+----

In [127]:
# Método 3

graph.bfs(fromExpr = "id = 'João Pessoa'", toExpr = "id = 'Cabedelo'", edgeFilter = "relationship = 'EROAD'").show()

+--------------------+--------------------+--------------------+--------------------+--------------------+
|                from|                  e0|                  v1|                  e1|                  to|
+--------------------+--------------------+--------------------+--------------------+--------------------+
|[João Pessoa, 52....|[João Pessoa, São...|[São José do rio ...|[São José do rio ...|[Cabedelo, 53.612...|
|[João Pessoa, 52....|[João Pessoa, Pat...|[Patos, 52.078663...|[Patos, Cabedelo,...|[Cabedelo, 53.612...|
+--------------------+--------------------+--------------------+--------------------+--------------------+



In [128]:
# Método 4

graph.bfs(fromExpr = "id = 'Campina Grande'", toExpr = "id = 'Mamanguape'", edgeFilter = "relationship = 'EROAD'").show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                from|                  e0|                  v1|                  e1|                  v2|                  e2|                  v3|                  e3|                  v4|                  e4|                  to|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|[Campina Grande, ...|[Campina Grande, ...|[João Pessoa, 52....|[João Pessoa, Rio...|[Rio Tinto, 51.50...|[Rio Tinto, Itapo...|[Itaporanga, 52.0...|[Itaporanga, Arei...|[Areia, 51.88921,...|[Areia, Mamanguap...|[Mamanguape, 51.9...|
+--------------------+--------------------+--------------------+----

In [129]:
# Método 5

graph.bfs(fromExpr = "id = 'Rio Tinto'", toExpr = "id = 'Areia'", edgeFilter = "relationship = 'EROAD'").show()

+--------------------+--------------------+--------------------+--------------------+--------------------+
|                from|                  e0|                  v1|                  e1|                  to|
+--------------------+--------------------+--------------------+--------------------+--------------------+
|[Rio Tinto, 51.50...|[Rio Tinto, Itapo...|[Itaporanga, 52.0...|[Itaporanga, Arei...|[Areia, 51.88921,...|
+--------------------+--------------------+--------------------+--------------------+--------------------+



## Execute 2 consultas utilizando o método find.

In [130]:
# Método Find 1)
graph.find("(a)-[relation]->(b); (a)-[relation2]->(c); (b)-[relation3]->(c)").show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                   a|            relation|                   b|           relation2|                   c|           relation3|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|[João Pessoa, 52....|[João Pessoa, Pat...|[Patos, 52.078663...|[João Pessoa, São...|[São José do rio ...|[Patos, São José ...|
|[João Pessoa, 52....|[João Pessoa, Pat...|[Patos, 52.078663...|[João Pessoa, Cam...|[Campina Grande, ...|[Patos, Campina G...|
|[Patos, 52.078663...|[Patos, São José ...|[São José do rio ...|[Patos, Campina G...|[Campina Grande, ...|[São José do rio ...|
|[Patos, 52.078663...|[Patos, São José ...|[São José do rio ...|[Patos, Cabedelo,...|[Cabedelo, 53.612...|[São José do rio ...|
|[João Pessoa, 52....|[João Pessoa, São...|[São José do rio ...|[João Pessoa, Cam...|[Campina Grande, ..

In [131]:
# Execute 2 consultas utilizando o método find.

# Método find 2)
graph.find("(a)-[relation]->(b); (a)-[relation2]->(c); (a)-[relation3]->(d); (a)-[relation4]->(e)").show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                   a|            relation|                   b|           relation2|                   c|           relation3|                   d|           relation4|                   e|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|[João Pessoa, 52....|[João Pessoa, Cam...|[Campina Grande, ...|[João Pessoa, São...|[São José do rio ...|[João Pessoa, São...|[São José do rio ...|[João Pessoa, São...|[São José do rio ...|
|[João Pessoa, 52....|[João Pessoa, Cam...|[Campina Grande, ...|[João Pessoa, São...|[São José do rio ...|[João Pessoa, São...|[São José do rio ...|[João Pessoa, Rio...|[Rio Tinto, 51.50...|
|[João Pessoa, 52....|[João Pessoa, Cam...|[C

## Execute 2 consultas utilizando o método filterVertices.

In [132]:
# Método filterVertices  1)
population = graph.vertices.filter("population > 100000").show()

+--------------+---------+---------+----------+
|            id| latitude|longitude|population|
+--------------+---------+---------+----------+
|   João Pessoa|52.379189| 4.899431|    809015|
|Campina Grande|52.092876| 5.104480|    334176|
|         Patos|52.078663| 4.288788|    107605|
+--------------+---------+---------+----------+



In [133]:
# Execute 2 consultas utilizando o método filterVertices.

# Método filterVertices  2)
graph.vertices.filter("id = 'Paraíba'").show()

+---+--------+---------+----------+
| id|latitude|longitude|population|
+---+--------+---------+----------+
+---+--------+---------+----------+



### Não retornou nada, pois Paraíba não está na coluna "id"

In [134]:
# Método filterVertices 3)
graph.vertices.filter("latitude > 51").show()

+--------------+---------+---------+----------+
|            id| latitude|longitude|population|
+--------------+---------+---------+----------+
|   João Pessoa|52.379189| 4.899431|    809015|
|Campina Grande|52.092876| 5.104480|    334176|
|         Patos|52.078663| 4.288788|    107605|
|      Cabedelo| 53.61239| -0.22219|      9642|
|     São Bento| 53.52285| -1.13116|     34031|
|    Itaporanga| 52.05917|  1.15545|     24692|
|    São Mamede| 52.01667|  4.70833|      7724|
+--------------+---------+---------+----------+



## Lista 6

## Determine o nó central através do grau.

In [180]:
degree = graph.degrees
degreein = graph.inDegrees
degreeout = graph.outDegrees

# Sem o parâmetro "how = 'left'", as cidades com valores "zero" em qualquer coluna, não aparecem.
(degree.join(degreein, "id", how = "left").join(degreeout, "id", how = "left").fillna(0).sort("Degree").show())

+--------------------+------+--------+---------+
|                  id|degree|inDegree|outDegree|
+--------------------+------+--------+---------+
|         Itaporanga |     1|       0|        1|
|             Triunfo|     2|       1|        1|
|           São Bento|     3|       2|        1|
|           Rio Tinto|     3|       2|        1|
|               Areia|     3|       2|        1|
|          Mamanguape|     3|       1|        2|
|          São Mamede|     4|       2|        2|
|            Cabedelo|     4|       4|        0|
|      Campina Grande|     4|       3|        1|
|São José do rio d...|     5|       3|        2|
|          Itaporanga|     5|       1|        4|
|               Patos|     5|       2|        3|
|         João Pessoa|     6|       1|        5|
+--------------------+------+--------+---------+



#### O nó central através do grau foi João Pessoa

## Determine o nó central por centralidade utilizando a distância de Wasserman e a distância harmônica (consultar material).

## Determine o nó central por intermediação.

In [181]:
pr = graph.pageRank(resetProbability = 0.15, maxIter = 50)
pr.vertices.sort("pagerank").show()

+--------------------+---------+---------+----------+------------------+
|                  id| latitude|longitude|population|          pagerank|
+--------------------+---------+---------+----------+------------------+
|             Triunfo|  51.9775|  4.13333|      9382|0.7715353710183569|
|               Patos|52.078663| 4.288788|    107605|0.8286005779465895|
|               Areia| 51.88921|  0.90421|     22819|0.8830738318958353|
|São José do rio d...|  51.9225|  4.47917|     18034|0.8893305800910256|
|           São Bento| 53.52285| -1.13116|     34031|0.9583106181082387|
|          Mamanguape| 51.96375|   1.3511|     44882|0.9771496551749105|
|      Campina Grande|52.092876| 5.104480|    334176| 1.026047723790365|
|           Rio Tinto|51.509865|-0.118092|     24172| 1.069117158772261|
|            Cabedelo| 53.61239| -0.22219|      9642|1.0805209777396108|
|         João Pessoa|52.379189| 4.899431|    809015| 1.098677475151878|
|          Itaporanga| 52.05917|  1.15545|     2469

#### O nó central por intermediação é o que obteve o melhor resultado no pagerank, no caso foi: São Mamede